In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%reload_ext autoreload
%autoreload 2

# np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
print("Total Params:", FRmodel.count_params())
def triplet_loss(y_true, y_pred, alpha = 0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.square(anchor-positive)
    neg_dist = tf.square(anchor-negative)
    basic_loss = tf.reduce_sum(pos_dist-neg_dist)+alpha
    loss = tf.reduce_sum(tf.maximum(basic_loss,0.))
    return loss
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    print("loss = " + str(loss.eval()))
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

W0102 19:45:33.360691  2000 deprecation_wrapper.py:119] From C:\Users\hrithik chinu\Miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0102 19:45:34.345257  2000 deprecation_wrapper.py:119] From C:\Users\hrithik chinu\Miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0102 19:45:35.297383  2000 deprecation_wrapper.py:119] From C:\Users\hrithik chinu\Miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0102 19:45:35.565299  2000 deprecation_wrapper.py:119] From C:\Users\hrithik chinu\Miniconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0102 19:45:35.56929

Total Params: 3743280


W0102 19:46:04.820718  2000 deprecation_wrapper.py:119] From C:\Users\hrithik chinu\Miniconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



loss = 350.02637


In [3]:
database = {}
database["cardi"] = img_to_encoding("images/girl_like_u/crop/cardi.jpg", FRmodel)
database["ellen"] = img_to_encoding("images/girl_like_u/crop/ellen.jpg", FRmodel)
database["adam"] = img_to_encoding("images/girl_like_u/crop/adam.jpg", FRmodel)
database["camila"] = img_to_encoding("images/girl_like_u/crop/camila.jpg", FRmodel)
database["jennifer_lopez"] = img_to_encoding("images/girl_like_u/crop/jennifer_lopez.jpg", FRmodel)
database["Gal"] = img_to_encoding("images/girl_like_u/crop/Gal.jpg", FRmodel)
database["Aly Raisman"] = img_to_encoding("images/girl_like_u/crop/Aly_Raisman.jpg", FRmodel)

In [ ]:
def cropFace(path,name):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = face_cascade.detectMultiScale(gray)
    for (x,y,w,h) in face:
        cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        cv2.putText(img = img, text = 'HEY HRITHIK!!!', org = (x+w, y+h), fontFace = cv2.FONT_HERSHEY_DUPLEX, fontScale = 0.5, 
                color = (0, 0, 255))
    cv2.imshow('face_detect_hrithik', roi_color)
    cv2.imwrite(name, roi_color)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    plt.imshow(roi_gray, cmap='gray', interpolation='bicubic')
    plt.show()

In [5]:
def who_is_it_live(image_path, database, model):
    encoding = img_to_encoding_ip_array(image_path, model)
    min_dist = 100
    for (name, db_enc) in database.items():
        dist = np.linalg.norm(encoding-database[name])
        if dist<min_dist:
            min_dist = dist
            identity = name
    if min_dist > 0.65:
#         print("Not in the database.")
        identity = 'Not present in DB'
#      else:
#         print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [7]:
cap = cv2.VideoCapture('images/girl_like_u/trim/adam.mp4')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
while(True):
    try:
        
        ret, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        face = face_cascade.detectMultiScale(gray)
        for (x,y,w,h) in face:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            dist, name = who_is_it_live(roi_color, database, FRmodel)
            if(name != 'Not present in DB'):
                cv2.rectangle(img, (x,y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(img = img, text = name+'{'+str(dist)+'}', org = (x+w, y+h), fontFace = cv2.FONT_HERSHEY_DUPLEX, fontScale = 1, color = (0, 255, 0), thickness=2)
        cv2.imshow('face_detectection', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except Exception as e:
        print(e)
        break
cap.release()
cv2.destroyAllWindows()